In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from torch import nn
import torch.nn.functional as F

import syft as sy
from syft import make_plan
from syft.lib.python.string import String
from syft import logger
from torch.utils.data import Dataset, DataLoader

from transformers import AutoTokenizer
import datasets
import tokenizers

# sy.lib.load('tokenizers')

In [3]:
# Create client

alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()
t = String("abcd")
tp = t.send(alice_client)

x = String("A simple sentence.")
x_ptr = x.send(alice_client)

In [4]:
# Load model

model_name = 'distilroberta-base'

# model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [5]:
# Tokenizer
# NOTE transformers tokenizer wraps a tokenizers.Tokenizer, we uwrap it.
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)._tokenizer
print(tokenizer)

In [7]:
type(tokenizer.to_str())

str

In [15]:
# Tokenizer serialization

from syft.proto.lib.python.string_pb2 import String as String_PB
from tokenizers import Tokenizer

def object2proto(obj: object) -> String_PB:
    # TODO Tokenizer.to_str serializes to a json string.
    # Protobuf has protobuf.json_format, which might be better than sending a raw string.
    protobuf_tokenizer = String_PB(data=obj.to_str())
    return protobuf_tokenizer


def proto2object(protobuf_tokenizer: String_PB) -> Tokenizer:
    tokenizer = Tokenizer.from_str(protobuf_tokenizer.data)
    return tokenizer

out = proto2object(object2proto(tokenizer))
assert out.to_str() == tokenizer.to_str()

In [9]:
# Encoding serialization

from syft.proto.lib.python.string_pb2 import String as String_PB
from tokenizers import Encoding

def object2proto(obj: object) -> String_PB:
    # TODO __getstate__ and __setstate__ return bytes, bytes protobuf?
    protobuf_obj = String_PB(data=obj.__getstate__().decode('utf-8'))
    return protobuf_obj


def proto2object(protobuf_encoding: String_PB) -> Encoding:
    encoding = Encoding()
    encoding.__setstate__(protobuf_encoding.data.encode('utf-8'))
    return encoding

encoding = tokenizer.encode("This is a test.")
assert encoding.__getstate__() == encoding.__getstate__()